In [1]:
from testing_criterions import CPTDigraphManager
from diblob import DigraphManager
import re

def extract_test_cases(test_cases, delimiter = ',', source='s', sink='t'):
    test_cases_string = ','.join([elem[0] for elem in test_cases])
    test_cases = test_cases_string.split(f'{sink},{source}')
    test_cases = [f"{source},{test_case.replace(source, '').replace(sink, '')[1:-1]},{sink}".replace(',', delimiter) 
                  for test_case in test_cases]
    return test_cases

def extract_k_test_cases(test_cases, delimiter = ',', source='s', sink='t', k=1):
    test_cases_string = ','.join([elem[0] for elem in test_cases]) + f",{source}"
    delimiters = "".join([f"{sink},{sink}{source}{number},{source}|" for number in range(k)])

    test_cases = re.split(delimiters[:-1], test_cases_string)
    test_cases = [test_case for test_case in test_cases if test_case]
    test_cases = [f"{source},{test_case.replace(source, '').replace(sink, '')[1:-1]},{sink}".replace(',', delimiter) 
                  for test_case in test_cases]
    return test_cases

In [3]:
# wariant 5:  minimalny sumaryczny koszt, liczba testow dowolna

def minimal_test_cost(digraph_manager, source='s', sink='t', cost={}):

    if source in digraph_manager or sink in digraph_manager:
        raise Exception('sink/source should be unique over digraph namespace!')
    
    dict_json_representation = dict(digraph_manager(digraph_manager.root_diblob_id))
    cpt = CPTDigraphManager(dict_json_representation, cost_function={})


    minimal_elements, maximal_elements = cpt.get_edge_elements()
    
    cpt.add_nodes(source, sink)

    minimal_edges = [(source, node_id) for node_id in minimal_elements]
    maximal_edges = [(node_id, sink) for node_id in maximal_elements]
    
    if not minimal_edges or not maximal_edges:
        raise Exception('Should be at leas minimal/maximal element in the diblob!')
    
    new_edges = minimal_edges + maximal_edges + [(sink, source)]

    cpt.connect_nodes(*new_edges)

    weight = len(cpt.nodes) * (len(cpt.nodes) + 1) / 2
    cost_function_update_dict = {edge_id: weight*cost.get(edge_id, 1) for edge_id in cpt.edges}
    cost_function_update_dict[(sink, source)] = 1

    cpt.update_cost_function(cost_function_update_dict)
    return cpt.compute_cpt(start_node=source)



In [4]:
cpt_digraph_manager = DigraphManager({"B0": {}})
cpt_digraph_manager.add_nodes('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I')
cpt_digraph_manager.connect_nodes(('A', 'C'), ('B', 'C'), ('C', 'D'), ('C', 'E'), ('C', 'F'),
                       ('D', 'I'), ('D', 'G'), ('E', 'G'), ('E', 'H'), ('F', 'H'))

test_cases, cost = minimal_test_cost(cpt_digraph_manager, source='S', sink='T')
print(extract_test_cases(test_cases, delimiter='|', source='S', sink='T'))

['S|B|C|E|G|T', 'S|A|C|D|G|T', 'S|A|C|D|I|T', 'S|A|C|F|H|T', 'S|A|C|E|H|T']


In [5]:
# wariant 1:  minimalna mozliwa liczba przypadkow testowych, kosztnie brany pod uwage
# ten sam kod co powyzej, zmieniona tylko waga (#TODO uwspólnic i dekorator)
def minimal_number_of_test_cases(digraph_manager, source='s', sink='t'):

    if source in digraph_manager or sink in digraph_manager:
        raise Exception('sink/source should be unique over digraph namespace!')
    
    dict_json_representation = dict(digraph_manager(digraph_manager.root_diblob_id))
    cpt = CPTDigraphManager(dict_json_representation, cost_function={})

    minimal_elements, maximal_elements = cpt.get_edge_elements()
    
    cpt.add_nodes(source, sink)

    minimal_edges = [(source, node_id) for node_id in minimal_elements]
    maximal_edges = [(node_id, sink) for node_id in maximal_elements]
    
    if not minimal_edges or not maximal_edges:
        raise Exception('Should be at leas minimal/maximal element in the diblob!')
    
    new_edges = minimal_edges + maximal_edges + [(sink, source)]

    cpt.connect_nodes(*new_edges)

    weight = len(cpt.nodes) * (len(cpt.nodes) + 1) / 2
    cost_function_update_dict = {edge_id: weight for edge_id in cpt.edges}
    cost_function_update_dict[(sink, source)] = weight ** 3

    cpt.update_cost_function(cost_function_update_dict)
    return cpt.compute_cpt(start_node=source)


In [6]:
cpt2_digraph_manager = DigraphManager({"B0": {}})
cpt2_digraph_manager.add_nodes('A', 'B', 'C', 'D', 'E', 'F', 'G')
cpt2_digraph_manager.connect_nodes(('A', 'B'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'F'),
                       ('D', 'F'), ('E', 'F'), ('F', 'B'), ('F', 'G'))

test_cases, _ = minimal_number_of_test_cases(cpt2_digraph_manager, source='S', sink='T')
print(extract_test_cases(test_cases, delimiter='|', source='S', sink='T'))

test_cases, _ = minimal_test_cost(cpt2_digraph_manager, source='S', sink='T', cost={('F', 'B'): 20})
print(extract_test_cases(test_cases, delimiter='|', source='S', sink='T'))

['S|A|B|C|F|B|E|F|B|D|F|G|T']
['S|A|B|C|F|G|T', 'S|A|B|E|F|B|D|F|G|T']


In [7]:
# wariant 4:  ustalona z gory liczba przypadkow testowych, minimalizacja kosztu calkowitego
def k_test_cases(digraph_manager, k = 1, source='s', sink='t'):

    if source in digraph_manager or sink in digraph_manager:
        raise Exception('sink/source should be unique over digraph namespace!')
    
    dict_json_representation = dict(digraph_manager(digraph_manager.root_diblob_id))
    cpt = CPTDigraphManager(dict_json_representation, cost_function={})

    minimal_elements, maximal_elements = cpt.get_edge_elements()
    
    nodes_to_add = [sink + source + str(number) for number in range(k)] + [sink, source]
    cpt.add_nodes(*nodes_to_add)

    minimal_edges = [(source, node_id) for node_id in minimal_elements]
    maximal_edges = [(node_id, sink) for node_id in maximal_elements]
    
    if not minimal_edges or not maximal_edges:
        raise Exception('Should be at least minimal/maximal element in the diblob!')
    
    sink_source_edges = [(sink, sink + source + str(number)) for number in range(k)] +\
                        [(sink + source + str(number), source) for number in range(k)]
    
    new_edges = minimal_edges + maximal_edges + sink_source_edges
    cpt.connect_nodes(*new_edges)

    print(cpt)
    weight = len(cpt.nodes) * (len(cpt.nodes) + 1) / 2
    cost_function_update_dict = {edge_id: weight for edge_id in cpt.edges}

    for number in range(k):
        cost_function_update_dict[(sink, sink + source + str(number))] = 1
        cost_function_update_dict[(sink + source + str(number), source)] = 1

    #cpt.update_cost_function(cost_function_update_dict)
    return cpt.compute_cpt(start_node=source)

In [9]:
cpt3_digraph_manager = DigraphManager({"B0": {}})
cpt3_digraph_manager.add_nodes('A', 'B', 'C', 'D', 'E', 'F', 'G')
cpt3_digraph_manager.connect_nodes(('A', 'B'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'F'),
                       ('D', 'F'), ('E', 'F'), ('F', 'B'), ('F', 'G'))

test_cases, _ = k_test_cases(cpt3_digraph_manager, source='S', sink='T', k=5)
print(extract_k_test_cases(test_cases, delimiter='|', source='S', sink='T', k=5))



{
"B0": {
    "F": ["B", "G"],
    "A": ["B"],
    "B": ["C", "D", "E"],
    "E": ["F"],
    "TS2": ["S"],
    "S": ["A"],
    "D": ["F"],
    "C": ["F"],
    "TS4": ["S"],
    "T": ["TS0", "TS1", "TS2", "TS3", "TS4"],
    "TS3": ["S"],
    "TS0": ["S"],
    "TS1": ["S"],
    "G": ["T"],
},
}

['S|A|B|C|F|G|T', 'S|A|B|D|F|G|T', 'S|A|B|D|F|G|T', 'S|A|B|D|F|G|T', 'S|A|B|E|F|B|D|F|G|T']
